<a href="https://colab.research.google.com/github/JustNoel05/POO-25A/blob/main/AeroFly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

David Yoel Aguilar Urenda Aerofly

In [ ]:
import datetime
import random
from typing import List, Dict, Optional

class Pasajero:
    def __init__(self, nombre: str, pasaporte: str, contacto: str):
        self.nombre = nombre
        self.pasaporte = pasaporte
        self.contacto = contacto

    def mostrar_informacion(self) -> str:
        return (f"Nombre: {self.nombre}\n"
                f"Pasaporte: {self.pasaporte}\n"
                f"Contacto: {self.contacto}")

class Vuelo:
    def __init__(self, destino: str, fecha_hora: datetime.datetime, numero_vuelo: str, asientos_totales: int):
        self.destino = destino
        self.fecha_hora = fecha_hora
        self.numero_vuelo = numero_vuelo
        self.asientos_totales = asientos_totales
        self.asientos_disponibles = asientos_totales
        self.pasajeros: List[Pasajero] = []

    def reservar_asiento(self, pasajero: Pasajero) -> bool:
        if self.asientos_disponibles > 0:
            self.pasajeros.append(pasajero)
            self.asientos_disponibles -= 1
            return True
        return False

    def cancelar_reserva(self, pasajero: Pasajero) -> bool:
        if pasajero in self.pasajeros:
            self.pasajeros.remove(pasajero)
            self.asientos_disponibles += 1
            return True
        return False

    def mostrar_informacion(self) -> str:
        return (f"Vuelo {self.numero_vuelo}\n"
                f"Destino: {self.destino}\n"
                f"Fecha y Hora: {self.fecha_hora.strftime('%Y-%m-%d %H:%M')}\n"
                f"Asientos disponibles: {self.asientos_disponibles}/{self.asientos_totales}")

class Reserva:
    def __init__(self, pasajero: Pasajero, vuelo: Vuelo, asientos_reservados: int):
        self.pasajero = pasajero
        self.vuelo = vuelo
        self.asientos_reservados = asientos_reservados
        self.codigo_reserva = self._generar_codigo()

    def _generar_codigo(self) -> str:
        return f"RES-{random.randint(1000, 9999)}-{self.vuelo.numero_vuelo}"

    def mostrar_informacion(self) -> str:
        return (f"Código de reserva: {self.codigo_reserva}\n"
                f"Pasajero: {self.pasajero.nombre}\n"
                f"Vuelo: {self.vuelo.numero_vuelo} a {self.vuelo.destino}\n"
                f"Fecha: {self.vuelo.fecha_hora.strftime('%Y-%m-%d')}\n"
                f"Asientos: {self.asientos_reservados}")

    def cancelar(self) -> bool:
        return self.vuelo.cancelar_reserva(self.pasajero)

class EuroFly:
    DIAS_OPERATIVOS = ["Monday", "Wednesday", "Friday"]
    HORARIOS_SALIDA = [datetime.time(5, 0), datetime.time(20, 0)]

    def __init__(self):
        self.nombre = "EuroFly"
        self.destinos: List[str] = []
        self.vuelos: List[Vuelo] = []
        self.reservas: List[Reserva] = []

    def agregar_destino(self, destino: str) -> None:
        if destino not in self.destinos:
            self.destinos.append(destino)

    def eliminar_destino(self, destino: str) -> bool:
        if destino in self.destinos:

            vuelos_destino = [v for v in self.vuelos if v.destino == destino and v.fecha_hora > datetime.datetime.now()]
            if vuelos_destino:
                return False
            self.destinos.remove(destino)
            return True
        return False

    def mostrar_destinos(self) -> str:
        return "\n".join(f"{i+1}. {destino}" for i, destino in enumerate(self.destinos))

    def crear_vuelo(self, destino: str, fecha: datetime.date, horario: datetime.time, asientos: int = 150) -> bool:

        if destino not in self.destinos:
            return False

        dia_semana = fecha.strftime("%A")
        if dia_semana not in self.DIAS_OPERATIVOS:
            return False

        if horario not in self.HORARIOS_SALIDA:
            return False

        fecha_hora = datetime.datetime.combine(fecha, horario)

        for vuelo in self.vuelos:
            if (vuelo.destino == destino and
                vuelo.fecha_hora == fecha_hora):
                return False

        numero_vuelo = f"EF-{random.randint(100, 999)}"


        nuevo_vuelo = Vuelo(destino, fecha_hora, numero_vuelo, asientos)
        self.vuelos.append(nuevo_vuelo)
        return True

    def buscar_vuelos(self, destino: Optional[str] = None, fecha: Optional[datetime.date] = None) -> List[Vuelo]:
        resultados = []
        ahora = datetime.datetime.now()

        for vuelo in self.vuelos:
            if vuelo.fecha_hora <= ahora:
                continue

            if destino and vuelo.destino != destino:
                continue
            if fecha and vuelo.fecha_hora.date() != fecha:
                continue

            resultados.append(vuelo)

        return resultados

    def cancelar_vuelo(self, numero_vuelo: str) -> bool:
        for vuelo in self.vuelos:
            if vuelo.numero_vuelo == numero_vuelo and vuelo.fecha_hora > datetime.datetime.now():
                reservas_a_cancelar = [r for r in self.reservas if r.vuelo == vuelo]
                for reserva in reservas_a_cancelar:
                    reserva.cancelar()
                    self.reservas.remove(reserva)

                self.vuelos.remove(vuelo)
                return True
        return False

    def hacer_reserva(self, pasajero: Pasajero, vuelo: Vuelo, asientos: int = 1) -> Optional[Reserva]:
        if vuelo not in self.vuelos or vuelo.fecha_hora <= datetime.datetime.now():
            return None

        if asientos <= 0 or asientos > vuelo.asientos_disponibles:
            return None

        for _ in range(asientos):
            if not vuelo.reservar_asiento(pasajero):
                return None

        nueva_reserva = Reserva(pasajero, vuelo, asientos)
        self.reservas.append(nueva_reserva)
        return nueva_reserva

    def cancelar_reserva(self, codigo_reserva: str) -> bool:
        for reserva in self.reservas:
            if reserva.codigo_reserva == codigo_reserva:
                if reserva.cancelar():
                    self.reservas.remove(reserva)
                    return True
        return False

def mostrar_menu_principal():
    print("\n--- Sistema de Gestión EuroFly ---")
    print("1. Gestión de Destinos")
    print("2. Gestión de Vuelos")
    print("3. Gestión de Reservas")
    print("4. Salir")

def mostrar_menu_destinos():
    print("\n--- Gestión de Destinos ---")
    print("1. Agregar destino")
    print("2. Eliminar destino")
    print("3. Mostrar destinos disponibles")
    print("4. Volver al menú principal")

def mostrar_menu_vuelos():
    print("\n--- Gestión de Vuelos ---")
    print("1. Crear nuevo vuelo")
    print("2. Buscar vuelos disponibles")
    print("3. Cancelar vuelo")
    print("4. Mostrar todos los vuelos")
    print("5. Volver al menú principal")

def mostrar_menu_reservas():
    print("\n--- Gestión de Reservas ---")
    print("1. Hacer nueva reserva")
    print("2. Cancelar reserva")
    print("3. Mostrar mis reservas")
    print("4. Volver al menú principal")

def main():
    aerolinea = EuroFly()
    aerolinea.agregar_destino("París")
    aerolinea.agregar_destino("Berlín")
    aerolinea.agregar_destino("Roma")
    aerolinea.agregar_destino("Madrid")
    aerolinea.agregar_destino("Lisboa")

    hoy = datetime.date.today()
    for i in range(5):
        fecha = hoy + datetime.timedelta(days=i*2)
        if fecha.strftime("%A") in EuroFly.DIAS_OPERATIVOS:
            aerolinea.crear_vuelo("París", fecha, EuroFly.HORARIOS_SALIDA[0])
            aerolinea.crear_vuelo("Berlín", fecha, EuroFly.HORARIOS_SALIDA[1])

    while True:
        mostrar_menu_principal()
        opcion = input("Seleccione una opción: ")

        if opcion == "1":
            while True:
                mostrar_menu_destinos()
                sub_opcion = input("Seleccione una opción: ")

                if sub_opcion == "1":
                    destino = input("Ingrese el nombre del nuevo destino: ").strip()
                    if destino:
                        aerolinea.agregar_destino(destino)
                        print(f"Destino '{destino}' agregado correctamente.")
                    else:
                        print("El nombre del destino no puede estar vacío.")

                elif sub_opcion == "2":
                    print("Destinos disponibles:")
                    print(aerolinea.mostrar_destinos())
                    try:
                        num = int(input("Ingrese el número del destino a eliminar (0 para cancelar): "))
                        if 0 < num <= len(aerolinea.destinos):
                            destino = aerolinea.destinos[num-1]
                            if aerolinea.eliminar_destino(destino):
                                print(f"Destino '{destino}' eliminado correctamente.")
                            else:
                                print("No se pudo eliminar el destino. Puede que tenga vuelos programados.")
                        elif num != 0:
                            print("Número inválido.")
                    except ValueError:
                        print("Por favor ingrese un número válido.")

                elif sub_opcion == "3":
                    print("\nDestinos disponibles:")
                    print(aerolinea.mostrar_destinos())

                elif sub_opcion == "4":
                    break

                else:
                    print("Opción no válida. Intente nuevamente.")

        elif opcion == "2":
            while True:
                mostrar_menu_vuelos()
                sub_opcion = input("Seleccione una opción: ")

                if sub_opcion == "1":
                    print("\nDestinos disponibles:")
                    print(aerolinea.mostrar_destinos())
                    try:
                        num_destino = int(input("Seleccione el número del destino: "))
                        if 0 < num_destino <= len(aerolinea.destinos):
                            destino = aerolinea.destinos[num_destino-1]

                            fecha_str = input("Ingrese la fecha del vuelo (YYYY-MM-DD): ")
                            try:
                                fecha = datetime.datetime.strptime(fecha_str, "%Y-%m-%d").date()
                                dia_semana = fecha.strftime("%A")

                                if dia_semana not in EuroFly.DIAS_OPERATIVOS:
                                    print(f"Error: EuroFly solo opera los {', '.join(EuroFly.DIAS_OPERATIVOS)}.")
                                    continue

                                print("Horarios disponibles:")
                                for i, horario in enumerate(EuroFly.HORARIOS_SALIDA):
                                    print(f"{i+1}. {horario.strftime('%H:%M')}")

                                num_horario = int(input("Seleccione el número del horario: "))
                                if 0 < num_horario <= len(EuroFly.HORARIOS_SALIDA):
                                    horario = EuroFly.HORARIOS_SALIDA[num_horario-1]

                                    asientos = input("Ingrese la cantidad de asientos (opcional, default 150): ")
                                    asientos = int(asientos) if asientos.isdigit() else 150

                                    if aerolinea.crear_vuelo(destino, fecha, horario, asientos):
                                        print("Vuelo creado exitosamente!")
                                    else:
                                        print("No se pudo crear el vuelo. Verifique los datos.")
                                else:
                                    print("Número de horario inválido.")
                            except ValueError:
                                print("Formato de fecha incorrecto. Use YYYY-MM-DD.")
                        else:
                            print("Número de destino inválido.")
                    except ValueError:
                        print("Por favor ingrese un número válido.")

                elif sub_opcion == "2":
                    print("\nBuscar vuelos:")
                    print("1. Buscar todos los vuelos disponibles")
                    print("2. Buscar por destino")
                    print("3. Buscar por fecha")
                    print("4. Buscar por destino y fecha")
                    filtro = input("Seleccione una opción de búsqueda: ")

                    destino = None
                    fecha = None

                    if filtro in ["2", "4"]:
                        print("\nDestinos disponibles:")
                        print(aerolinea.mostrar_destinos())
                        try:
                            num = int(input("Seleccione el número del destino: "))
                            if 0 < num <= len(aerolinea.destinos):
                                destino = aerolinea.destinos[num-1]
                            else:
                                print("Número inválido.")
                                continue
                        except ValueError:
                            print("Por favor ingrese un número válido.")
                            continue

                    if filtro in ["3", "4"]:
                        fecha_str = input("Ingrese la fecha (YYYY-MM-DD): ")
                        try:
                            fecha = datetime.datetime.strptime(fecha_str, "%Y-%m-%d").date()
                        except ValueError:
                            print("Formato de fecha incorrecto. Use YYYY-MM-DD.")
                            continue

                    vuelos = aerolinea.buscar_vuelos(destino, fecha)
                    if vuelos:
                        print("\nVuelos encontrados:")
                        for i, vuelo in enumerate(vuelos, 1):
                            print(f"{i}. {vuelo.mostrar_informacion()}")
                            print("-" * 40)
                    else:
                        print("No se encontraron vuelos con los criterios especificados.")

                elif sub_opcion == "3":
                    vuelos_futuros = [v for v in aerolinea.vuelos if v.fecha_hora > datetime.datetime.now()]
                    if vuelos_futuros:
                        print("\nVuelos disponibles para cancelar:")
                        for i, vuelo in enumerate(vuelos_futuros, 1):
                            print(f"{i}. {vuelo.mostrar_informacion()}")
                            print("-" * 40)

                        try:
                            num = int(input("Seleccione el número del vuelo a cancelar (0 para cancelar): "))
                            if 0 < num <= len(vuelos_futuros):
                                vuelo = vuelos_futuros[num-1]
                                if aerolinea.cancelar_vuelo(vuelo.numero_vuelo):
                                    print("Vuelo cancelado exitosamente!")
                                else:
                                    print("No se pudo cancelar el vuelo.")
                            elif num != 0:
                                print("Número inválido.")
                        except ValueError:
                            print("Por favor ingrese un número válido.")
                    else:
                        print("No hay vuelos futuros para cancelar.")

                elif sub_opcion == "4":
                    vuelos_ordenados = sorted(aerolinea.vuelos, key=lambda v: v.fecha_hora)
                    if vuelos_ordenados:
                        print("\nTodos los vuelos:")
                        for vuelo in vuelos_ordenados:
                            estado = "PASADO" if vuelo.fecha_hora <= datetime.datetime.now() else "FUTURO"
                            print(f"[{estado}] {vuelo.mostrar_informacion()}")
                            print("-" * 40)
                    else:
                        print("No hay vuelos registrados.")

                elif sub_opcion == "5":
                    break

                else:
                    print("Opción no válida. Intente nuevamente.")

        elif opcion == "3":
            while True:
                mostrar_menu_reservas()
                sub_opcion = input("Seleccione una opción: ")

                if sub_opcion == "1":
                    print("\nInformación del pasajero:")
                    nombre = input("Nombre completo: ").strip()
                    pasaporte = input("Número de pasaporte: ").strip()
                    contacto = input("Información de contacto: ").strip()

                    if not nombre or not pasaporte:
                        print("Nombre y pasaporte son obligatorios.")
                        continue

                    pasajero = Pasajero(nombre, pasaporte, contacto)

                    print("\nBuscar vuelos para reserva:")
                    vuelos_disponibles = aerolinea.buscar_vuelos()
                    if not vuelos_disponibles:
                        print("No hay vuelos disponibles para reservar.")
                        continue

                    print("\nVuelos disponibles:")
                    for i, vuelo in enumerate(vuelos_disponibles, 1):
                        print(f"{i}. {vuelo.mostrar_informacion()}")
                        print("-" * 40)

                    try:
                        num_vuelo = int(input("Seleccione el número del vuelo: "))
                        if 0 < num_vuelo <= len(vuelos_disponibles):
                            vuelo = vuelos_disponibles[num_vuelo-1]

                            asientos = int(input("Número de asientos a reservar: "))
                            if asientos <= 0:
                                print("Debe reservar al menos 1 asiento.")
                                continue

                            reserva = aerolinea.hacer_reserva(pasajero, vuelo, asientos)
                            if reserva:
                                print("\nReserva creada exitosamente!")
                                print(reserva.mostrar_informacion())
                            else:
                                print("No se pudo completar la reserva. Puede que no haya suficientes asientos disponibles.")
                        else:
                            print("Número de vuelo inválido.")
                    except ValueError:
                        print("Por favor ingrese un número válido.")

                elif sub_opcion == "2":
                    if not aerolinea.reservas:
                        print("No hay reservas registradas.")
                        continue

                    print("\nReservas existentes:")
                    for i, reserva in enumerate(aerolinea.reservas, 1):
                        print(f"{i}. {reserva.mostrar_informacion()}")
                        print("-" * 40)

                    try:
                        num = int(input("Seleccione el número de reserva a cancelar (0 para cancelar): "))
                        if 0 < num <= len(aerolinea.reservas):
                            codigo = aerolinea.reservas[num-1].codigo_reserva
                            if aerolinea.cancelar_reserva(codigo):
                                print("Reserva cancelada exitosamente!")
                            else:
                                print("No se pudo cancelar la reserva.")
                        elif num != 0:
                            print("Número inválido.")
                    except ValueError:
                        print("Por favor ingrese un número válido.")

                elif sub_opcion == "3":
                    if not aerolinea.reservas:
                        print("No hay reservas registradas.")
                        continue

                    print("\nTodas las reservas:")
                    for reserva in aerolinea.reservas:
                        print(reserva.mostrar_informacion())
                        print("-" * 40)

                elif sub_opcion == "4":
                    break

                else:
                    print("Opción no válida. Intente nuevamente.")

        elif opcion == "4":
            print("Gracias por usar el sistema de EuroFly. ¡Hasta pronto!")
            break

        else:
            print("Opción no válida. Intente nuevamente.")

if __name__ == "__main__":
    main()


--- Sistema de Gestión EuroFly ---
1. Gestión de Destinos
2. Gestión de Vuelos
3. Gestión de Reservas
4. Salir
Seleccione una opción: 3}
Opción no válida. Intente nuevamente.

--- Sistema de Gestión EuroFly ---
1. Gestión de Destinos
2. Gestión de Vuelos
3. Gestión de Reservas
4. Salir
Seleccione una opción: 3

--- Gestión de Reservas ---
1. Hacer nueva reserva
2. Cancelar reserva
3. Mostrar mis reservas
4. Volver al menú principal
Seleccione una opción: 3
No hay reservas registradas.

--- Gestión de Reservas ---
1. Hacer nueva reserva
2. Cancelar reserva
3. Mostrar mis reservas
4. Volver al menú principal
Seleccione una opción: 1

Información del pasajero:
Nombre completo: David Yoel Aguilar Urenda
Número de pasaporte: 123921834
Información de contacto: 3338078916

Buscar vuelos para reserva:

Vuelos disponibles:
1. Vuelo EF-359
Destino: París
Fecha y Hora: 2025-04-07 05:00
Asientos disponibles: 150/150
----------------------------------------
2. Vuelo EF-512
Destino: Berlín
Fecha y 